<a href="https://colab.research.google.com/github/sbhattac/ai-workshop/blob/master/ANN/AI_Workshop_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Networks (ANNs)

### A Team Project to:
* Introduce basic concepts of ANNs 
* Demonstrate set of computational techniques inspired by natural neural systems
* To simulate or model natural systems with ANNs   

<p><em><strong>Attribution: This jupyter notebook builds upon material found at <a href="https://cspogil.org/Home">https://cspogil.org/Home</a> </strong></em></p>
<p>&nbsp;</p>
<p><a href="http://creativecommons.org/licenses/by-nc-sa/4.0/" rel="license"><img style="border-width: 0;" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" alt="Creative Commons License" /></a><br />This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/" rel="license">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.</p>


In [ ]:
#@title # Assigning Roles and Responsibilities within each group

#@markdown ---
#@markdown ### Enter Instructor Name:
Instructor_Name = "" #@param {type:"string"}
#@markdown 1. Introduces activities. Assigns roles to participants.
#@markdown 2. Responds for help or clarification request.
#@markdown 3. Collects the Jupiter notebooks from Recorder and Evaluator
#@markdown ---

#@markdown ### Enter Facilitator Name:
Facilitator_Name = "" #@param {type:"string"} 
#@markdown ### Enter Backup Facilitator Name:
Backup_Facilitator_Name = "" #@param {type:"string"} 
#@markdown 1.	Reads aloud each question and ask for volunteers to answer. If there is no volunteer then he/she starts the discussion and asks one participant after another for comments, solutions, answers, or clarifications.  When majority participants agree then she/he asks Recorder to record the answer. Also coordinates discussion about the code execution and the output like any other question.   
#@markdown 2.	Involves each participant equally in the discussions.    
#@markdown 3.	Turn the coordinating role to Evaluator after finishing each activity.  
#@markdown ---

#@markdown ### Enter Recorder Name:
Recorder_Name = "" #@param {type:"string"}
#@markdown ### Enter Backup Recorder Name:
Backup_Recorder_Name = "" #@param {type:"string"}  
#@markdown 1.	Coordinates Zoom screen access. Displays his/her screen when asking questions. Gives access to screen sharing as requested    
#@markdown 2.	Records all answers  for each question  inside the Jupiter Notebook   
#@markdown 3. Use "Run all" in menu "Runtime" and then "Save" Jupiter Workbook with all answers and results.
#@markdown 4. Submit Jupiter notebook with all answers and results of the running code.
#@markdown ---

#@markdown ### Enter Evaluator Name:
Evaluator_Name = "" #@param {type:"string"}
#@markdown ### Enter Backup Evaluator Name:
Backup_Evaluator_Name = "" #@param {type:"string"} 
#@markdown 1.	Keeps track of time for each designated Activity.   
#@markdown 2.	After each activity leads the discussion about material and collects feedback in the form of the table below.   
#@markdown 3. Submit Jupiter notebook with all comments and results of discussion at the end of each activity.  
#@markdown ---

#@markdown ### Enter Participant names
Participant_4_Name = "" #@param {type:"string"} 
Participant_5_Name = "" #@param {type:"string"}
Participant_6_Name = "" #@param {type:"string"}
Participant_7_Name = "" #@param {type:"string"}
Participant_8_Name = "" #@param {type:"string"}

#@markdown 1.	Participates actively in team work to answer all questions.
#@markdown 2.	Executes the code and shares the comments.
#@markdown ---


In [ ]:
#@title (RUN CELL) Download files to work on and install necessary libraries
!pip install palettable

In [ ]:
#@title (RUN CELL) Library Imports
from matplotlib import pyplot
import matplotlib.patches as patches
from math import cos, sin, atan
from palettable.tableau import Tableau_10
from time import localtime, strftime
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title (RUN CELL) Helper Classes and Functions for ANN visualization

# This code is a modified version of the libray written by Jianzheng Liu found at https://github.com/jzliu-100/visualize-neural-network.git  

from matplotlib import pyplot
import matplotlib.patches as patches
from math import cos, sin, atan
from palettable.tableau import Tableau_10
from time import localtime, strftime
import numpy as np

class Neuron():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, neuron_radius, id=-1):
        circle = pyplot.Circle((self.x, self.y), radius=neuron_radius, fill=False)
        pyplot.gca().add_patch(circle)
        pyplot.gca().text(self.x, self.y-0.15, str(id), size=10, ha='center')

class Layer():
    def __init__(self, network, number_of_neurons, number_of_neurons_in_widest_layer):
        self.vertical_distance_between_layers = 6
        self.horizontal_distance_between_neurons = 2
        self.neuron_radius = 0.5
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.previous_layer = self.__get_previous_layer(network)
        self.y = self.__calculate_layer_y_position()
        self.neurons = self.__intialise_neurons(number_of_neurons)

    def __intialise_neurons(self, number_of_neurons):
        neurons = []
        x = self.__calculate_left_margin_so_layer_is_centered(number_of_neurons)
        for iteration in range(number_of_neurons):
            neuron = Neuron(x, self.y)
            neurons.append(neuron)
            x += self.horizontal_distance_between_neurons
        return neurons

    def __calculate_left_margin_so_layer_is_centered(self, number_of_neurons):
        return self.horizontal_distance_between_neurons * (self.number_of_neurons_in_widest_layer - number_of_neurons) / 2

    def __calculate_layer_y_position(self):
        if self.previous_layer:
            return self.previous_layer.y + self.vertical_distance_between_layers
        else:
            return 0

    def __get_previous_layer(self, network):
        if len(network.layers) > 0:
            return network.layers[-1]
        else:
            return None

    def __line_between_two_neurons(self, neuron1, neuron2, i,j, weight=0.4, textoverlaphandler=None, show_weights=True):
        angle = atan((neuron2.x - neuron1.x) / float(neuron2.y - neuron1.y))
        x_adjustment = self.neuron_radius * sin(angle)
        y_adjustment = self.neuron_radius * cos(angle)

        # assign colors to lines depending on the sign of the weight
        color=Tableau_10.mpl_colors[0]
        if weight > 0: color=Tableau_10.mpl_colors[1]

        # assign different linewidths to lines depending on the size of the weight
        abs_weight = abs(weight)        
        if abs_weight > 0.5: 
            linewidth = 10*abs_weight
        elif abs_weight > 0.8: 
            linewidth =  100*abs_weight
        else:
            linewidth = abs_weight

        # draw the weights and adjust the labels of weights to avoid overlapping
        if True: #abs_weight > 0.5: 
            # while loop to determine the optimal locaton for text lables to avoid overlapping
            index_step = 2
            num_segments = 10   
            txt_x_pos = neuron1.x - x_adjustment+index_step*(neuron2.x-neuron1.x+2*x_adjustment)/num_segments
            txt_y_pos = neuron1.y - y_adjustment+index_step*(neuron2.y-neuron1.y+2*y_adjustment)/num_segments
            while ((not textoverlaphandler.getspace([txt_x_pos-0.5, txt_y_pos-0.5, txt_x_pos+0.5, txt_y_pos+0.5])) and index_step < num_segments):
                index_step = index_step + 1
                txt_x_pos = neuron1.x - x_adjustment+index_step*(neuron2.x-neuron1.x+2*x_adjustment)/num_segments
                txt_y_pos = neuron1.y - y_adjustment+index_step*(neuron2.y-neuron1.y+2*y_adjustment)/num_segments

            # print("Label positions: ", "{:.2f}".format(txt_x_pos), "{:.2f}".format(txt_y_pos), "{:3.2f}".format(weight))
            # a=pyplot.gca().text(txt_x_pos, txt_y_pos, "{:3.2f}".format(weight), size=8, ha='center')
            if show_weights:
                a=pyplot.gca().text(txt_x_pos, txt_y_pos, "w"+str(i+1)+str(j+1), size=8, ha='left',fontsize=12, color='red')
                a.set_bbox(dict(facecolor='white', alpha=0))
            # print(a.get_bbox_patch().get_height())

        line = pyplot.Line2D((neuron1.x - x_adjustment, neuron2.x + x_adjustment), (neuron1.y - y_adjustment, neuron2.y + y_adjustment), linewidth=linewidth, color=color)
        x1,x2 = neuron1.x - x_adjustment, neuron2.x + x_adjustment
        y1,y2 = neuron1.y - y_adjustment, neuron2.y + y_adjustment
        pyplot.annotate("",
                xy=(x1, y1), xycoords='data',
                xytext=(x2, y2), textcoords='data',
                arrowprops=dict(arrowstyle="->", color="0",
                                shrinkA=5, shrinkB=5,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=0.",
                                ),
                )
        #pyplot.gca().add_line(line)

    def draw(self, layerType=0, weights=None, textoverlaphandler=None, show_weights=True):
        j=0 # index for neurons in this layer
        for neuron in self.neurons:            
            i=0 # index for neurons in previous layer
            neuron.draw( self.neuron_radius, id=j+1 )
            if self.previous_layer:
                for previous_layer_neuron in self.previous_layer.neurons:
                    self.__line_between_two_neurons(neuron, previous_layer_neuron, i,j, weights[i,j], textoverlaphandler, show_weights=show_weights)
                    i=i+1
            j=j+1
        
        # write Text
        x_text = self.number_of_neurons_in_widest_layer * self.horizontal_distance_between_neurons
        if layerType == 0:
            pyplot.text(x_text, self.y, 'Input Layer', fontsize = 12)
        elif layerType == -1:
            pyplot.text(x_text, self.y, 'Output Layer', fontsize = 12)
        else:
            pyplot.text(x_text, self.y, 'Hidden Layer '+str(layerType), fontsize = 12)

# A class to handle Text Overlapping
# The idea is to first create a grid space, if a grid is already occupied, then
# the grid is not available for text labels.
class TextOverlappingHandler():
    # initialize the class with the width and height of the plot area
    def __init__(self, width, height, grid_size=0.2):
        self.grid_size = grid_size
        self.cells = np.ones((int(np.ceil(width / grid_size)), int(np.ceil(height / grid_size))), dtype=bool)

    # input test_coordinates(bottom left and top right), 
    # getspace will tell you whether a text label can be put in the test coordinates
    def getspace(self, test_coordinates):
        x_left_pos = int(np.floor(test_coordinates[0]/self.grid_size))
        y_botttom_pos = int(np.floor(test_coordinates[1]/self.grid_size))
        x_right_pos = int(np.floor(test_coordinates[2]/self.grid_size))
        y_top_pos = int(np.floor(test_coordinates[3]/self.grid_size))
        if self.cells[x_left_pos, y_botttom_pos] and self.cells[x_left_pos, y_top_pos] \
        and self.cells[x_right_pos, y_top_pos] and self.cells[x_right_pos, y_botttom_pos]:
            for i in range(x_left_pos, x_right_pos):
                for j in range(y_botttom_pos, y_top_pos):
                    self.cells[i, j] = False

            return True
        else:
            return False

class NeuralNetwork():
    def __init__(self, number_of_neurons_in_widest_layer):
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.layers = []
        self.layertype = 0

    def add_layer(self, number_of_neurons ):
        layer = Layer(self, number_of_neurons, self.number_of_neurons_in_widest_layer)
        self.layers.append(layer)

    def draw(self, weights_list=None, show_weights=True):
        # vertical_distance_between_layers and horizontal_distance_between_neurons are the same with the variables of the same name in layer class
        vertical_distance_between_layers = 6
        horizontal_distance_between_neurons = 2
        overlaphandler = TextOverlappingHandler(\
            self.number_of_neurons_in_widest_layer*horizontal_distance_between_neurons,\
            len(self.layers)*vertical_distance_between_layers, grid_size=0.2 )

        pyplot.figure(figsize=(12, 9))
        for i in range( len(self.layers) ):
            layer = self.layers[i]                                
            if i == 0:
                layer.draw( layerType=0, show_weights=show_weights)
            elif i == len(self.layers)-1:
                layer.draw( layerType=-1, weights=weights_list[i-1], textoverlaphandler=overlaphandler, show_weights=show_weights)
            else:
                layer.draw( layerType=i, weights=weights_list[i-1], textoverlaphandler=overlaphandler, show_weights=show_weights)

        pyplot.axis('scaled')
        pyplot.axis('off')
        pyplot.title( 'Neural Network architecture', fontsize=15 )
        #figureName='ANN_'+strftime("%Y%m%d_%H%M%S", localtime())+'.png'
        #pyplot.savefig(figureName, dpi=300, bbox_inches="tight")
        pyplot.show()

class ANN_Arch_View():
    # para: neural_network is an array of the number of neurons 
    # from input layer to output layer, e.g., a neural network of 5 nerons in the input layer, 
    # 10 neurons in the hidden layer 1 and 1 neuron in the output layer is [5, 10, 1]
    # para: weights_list (optional) is the output weights list of a neural network which can be obtained via classifier.coefs_
    def __init__( self, neural_network, weights_list=None ):
        self.neural_network = neural_network
        self.weights_list = weights_list
        # if weights_list is none, then create a uniform list to fill the weights_list
        if weights_list is None:
            weights_list=[]
            for first, second in zip(neural_network, neural_network[1:]):
                tempArr = np.ones((first, second))*0.4
                weights_list.append(tempArr)
            self.weights_list = weights_list
        
    def draw( self, show_weights=True):
        widest_layer = max( self.neural_network )
        network = NeuralNetwork( widest_layer )
        for l in self.neural_network:
            if l>0:
                network.add_layer(l)
        network.draw(self.weights_list,show_weights=show_weights)

# Activity 1. Biological Nervous Systems

## Activity 1 Scientific Basis of ANNs:

In (most) natural systems, information processing is done by the nervous​ system, which consists of the brain, spinal cord, and peripheral nerves. Each of these parts contains many nerve​ ​cells​, also called neurons​. The table above shows the typical number of neurons in various animal species. Describe the relationship between number of neurons and the animal’s:

# Fig 1

Consider the following diagram in Fig. 1, and answer questions 1-2:

![ANN fig 1](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig1.png)


In [ ]:
#@title #(RUN upon completion) Activity 1 Questions 1 - 2
#@markdown 1. Describe the relationship between number of neurons and the animal’s size
activity1_answer1 = "" #@param {type:"string"}
#@markdown 2. Describe the relationship between number of neurons and the animal’s intelligence
activity1_answer2 = "" #@param {type:"string"}

# Fig 2
Neurons can be grouped into 3 broad categories:

● <b>afferent neurons</b> send signals toward the brain

● <b>efferent neurons</b> send signals away from the brain

● <b>interneurons connect</b> other neurons.

(Remember that <b>afferents approach</b> the brain,
and <b>efferents exit</b> the brain.)

Answer questions 3-11 based on these terms. 

![ANN fig 2](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig2.png)


In [ ]:
#@title #(RUN upon completion) Activity 1 Questions 3 - 11
#@markdown ###Label each of the following as A (afferent) or E (efferent) :

#@markdown 3.	Photoreceptor (light-sensitive) cells in the retina of the eye
activity1_answer3 = "" #@param {type:"string"}
#@markdown 4. Hair cells that react to sound vibrations in the cochlea of the ear
activity1_answer4 = "" #@param {type:"string"}
#@markdown 5. Cells in muscles that cause the muscles to move
activity1_answer5 = "" #@param {type:"string"}
#@markdown 6.	Cells in muscles that sense the relative position of body parts (used for proprioception)
activity1_answer6 = "" #@param {type:"string"}
#@markdown 7.	Cells that cause the mouth to produce saliva
activity1_answer7 = "" #@param {type:"string"}

#@markdown ###Sensory neurons sense information, and motor neurons control muscles and glands.
#@markdown 8.  Which examples above (3-7) are sensory neurons?
activity1_answer8 = "" #@param {type:"string"}
#@markdown 9.  Which examples above (3-7) are motor neurons?
activity1_answer9 = "" #@param {type:"string"}
#@markdown 10.  Are sensory neurons afferent or efferent?
activity1_answer10 = "" #@param {type:"string"}
#@markdown 11.  Are motor neurons afferent or efferent?
activity1_answer11 = "" #@param {type:"string"}

# Fig 3
Different types of neurons have different shapes and structures, but most contain a similar set of components, shown below.

When a neuron <b>fires</b>, a electrical signal travels from the soma, through the axon, and to terminals that connect to the dendrites of other neurons, so that some of these other neurons may also fire as a result of this signal. This signal is called an <b>action potential</b> and is how the system <b>reacts</b>. 

The dendritic connections between neurons change over time (more slowly). These changing connections are how the system <b>learns</b>. 

To create artificial neural systems (either as simulations or to solve other problems), we need to consider both processes - <b>reacting</b> and <b>learning</b>.

Consider the following diagram in Fig. 3, and answer questions 12-15:

![ANN fig 3 ](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig3-neuron-image.png)


In [ ]:
#@title #(RUN upon completion) Activity 1 Questions 12 - 15
#@markdown ###Match the labels (A-D) to the descriptions below:

#@markdown 12.	The soma is the main cell body with the nucleus .
activity1_answer3 = "" #@param {type:"string"}
#@markdown 13. The soma has branching dendrites that receive signals from other cells.
activity1_answer4 = "" #@param {type:"string"}
#@markdown 14. The axon is the long arm that sends signals from the soma.
activity1_answer5 = "" #@param {type:"string"}
#@markdown 15.	Terminals connect the axon to the dendrites of other neurons.
activity1_answer6 = "" #@param {type:"string"}

# Activity 2. Perceptrons




## A) Bridging the gap between biological and artificial NNs: Perceptron

In a typical Artificial Neural Network (ANN), each neuron has a set of <b>inputs</b> and one <b>output</b>. Each input value is multipled by a <b>weight</b> (positive or negative) to determine its effect. 

The weighted inputs are added together, and then an <b>activation function</b> (also called a <b>transfer function</b>) is applied to the sum to determine the output value. 

![ANN fig 4 ](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig4-perceptron.png)

Thus the neuron’s operation can be written as:
$$y_j=f(\sum_{i=1}^n x_iw_{ij})$$

This model of a neuron is called a <b>perceptron</b>.

In [ ]:
#@title #(RUN upon completion) Activity 2 (A) Questions 1 - 7
#@markdown ###Refer to the diagram and equation above and identify:
#@markdown 1.	the letter used for the <b>input(s)</b> ?
activity2_answer1 = "" #@param {type:"string"} 
#@markdown 2.	the letter used for the <b>output(s)</b> ?
activity2_answer2 = "" #@param {type:"string"} 
#@markdown 3.	the letter used for the <b>transfer function</b> ?
activity2_answer3 = "" #@param {type:"string"} 
#@markdown 4.	the letter & subscripts used for the <b>weight</b> between input #2 and output #1.
activity2_answer4 = "" #@param {type:"string"} 
#@markdown 5.	the letter that best fits a neuron’s <b>dendrites</b> ?
activity2_answer5 = "" #@param {type:"string"} 
#@markdown 6.	the letter that best fits a neuron’s <b>terminals</b> ?
activity2_answer6 = "" #@param {type:"string"} 
#@markdown 7.	the letter that best fits a neuron’s <b>soma</b> ?
activity2_answer7 = "" #@param {type:"string"} 

## B) Modifying code to visualize different ANN architectures:
The code cell below when executed will show you the architecture of an ANN with some small differences with the picture shown above. The inputs also have their own nodes (circles). Note that they <b> do not </b> apply an activation function. All other nodes apply activation function on their weighted inputs. Also, note that the inputs are just labeled with numbers and not with 'x' and a subscript. The output y is also understood and not shown. Only weights are shown.  

<b>This code cell below needs to be "shown" by choosing Edit -> Show/hide code. You are asked to change some values in it and re-execute it </b> in questions 8 - 12 below. After this work you can hide it again by choosing Edit -> Show/hide code.

In Python comments begin with # and those lines with comments that have the words <b>TRY CHANGING</b> are usually the ones you will modify in the exercises.

In [ ]:
#@title # (MODIFY AND RUN CELL) Coding for Activity 2 (B) to answer Questions 8 - 10
network=ANN_Arch_View([2,1]) # TRY CHANGING, the first number in this list is number of input nodes, next number is number of output nodes
network.draw(show_weights=True)

In [ ]:
#@title # Activity 2 Questions 8 - 10
#@markdown 8.	Change the code cell above and run it to visualize an ANN with 5 input nodes and 1 output node. What is the name of the weight connecting input node 4 to the output node?
activity2_answer2 = "" #@param {type:"string"} 
#@markdown 9.	Change the code cell above and run it to visualize an ANN with 5 input nodes and 3 output nodes. What is the name of the weight connecting input node 2 to the output node 3?
activity2_answer3 = "" #@param {type:"string"} 
#@markdown 10. How many weight values will an ANN of 7 input nodes and 5 output nodes have? You may change the code and run it to visualize this ANN.
activity2_answer3 = "" #@param {type:"string"} 

In [ ]:
#@title # (MODIFY AND RUN CELL) Coding for Activity 2 (B) to answer Questions 11 - 13
network=ANN_Arch_View([4, 2, 2, 1]) # TRY CHANGING, list of number of nodes in all layers starting from input through hidden layers and ending with number of nodes in output layer
network.draw(show_weights=True)

In [ ]:
#@title #(RUN upon completion) Activity 2 (B) Questions 11 - 13
#@markdown 11. Why are some layers called hidden layers as seen in the ANN architecture above? Hint: think of an approximate analogy to biological nervous systems.
activity2_answer11 = "" #@param {type:"string"} 
#@markdown 12. An ANN of architecture <b>[4, 2, 2, 1]</b> has two weights named <b>w22</b> - are they the same?
activity2_answer12 = "" #@param {type:"string"} 
#@markdown 13. How many weights would an ANN of architecture <b>[5, 4, 3, 2, 1]</b> have? You may change and re-run the code cell above to visualize it. <b>Extra fun</b>: to unclutter the picture you can hide the weight names by setting the show_weights input to False, with show_weights=False (just change True to False)
activity2_answer13 = "" #@param {type:"string"} 

# Activity 3. Perceptrons and Activation Functions



In [ ]:
#@title #(RUN CELL) Class and Object for a [3,1] ANN creation in PyTorch

class MyANNModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(3, 1,bias=False)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.fc1(x)
    x = my_activation(x)
    return x
    
ANN_model = MyANNModel()

In [ ]:
#@title #(RUN CELL) Helper Functions to create widgets for controlling the [3,1] PyTorch ANN

def plot_output_node_activation(model, num_input_nodes, out_node_num, **k):
    '''Plots the activation output of one node given a model which has no hidden layers'''
    plt.figure(figsize=(7,5))
    plt.title('Output of activation function from node ' + str(out_node_num))
    plt.ylim(-1.2, 1.2)
    plt.xlim(-2, 2)
    wsums = np.arange(-2,2,0.1)
    pl=plt.plot(wsums,my_activation(torch.tensor(wsums)))
    x_vals = []
    w_vals = []
    for i in range(num_input_nodes):
        x_vals.append(k['x'+str(i+1)])
        w_vals.append(k['w'+str(i+1)+str(out_node_num)])
        model.fc1.__dict__['_parameters']['weight'][out_node_num-1][i] = k['w'+str(i+1)+str(out_node_num)]
    xv = np.dot(x_vals,w_vals)
    yv = model.forward(torch.tensor([x_vals])).item()
    plt.scatter([xv], [yv], color='red', s=30)
    plt.xlabel('weighted sum of inputs')
    plt.ylabel('activation function output')
    plt.grid(True)
    plt.tight_layout() 

def make_output_node_widgets(model, num_input_nodes, out_node_num):
    wght_wdgts = [] # weight widgets
    inpt_wdgts = [] # input widgets
    for i in range(1,(num_input_nodes+1)):
        inpt_wdgts.append(widgets.FloatSlider(min=-1.0,max=1.0,description='x'+str(i)))
        wght_wdgts.append(widgets.FloatSlider(min=0.0,max=1.0,description='w'+str(i)+str(out_node_num)))
    ui = widgets.HBox([widgets.VBox(inpt_wdgts), widgets.VBox(wght_wdgts)])
    all_wgts ={}
    all_wgts['out_node_num'] = widgets.fixed(out_node_num)
    all_wgts['num_input_nodes'] = widgets.fixed(num_input_nodes)
    all_wgts['model'] = widgets.fixed(model)
    for xws in inpt_wdgts:
        all_wgts[xws.description] = xws
    for wws in wght_wdgts:
        all_wgts[wws.description] = wws
    inter = widgets.interactive_output(plot_output_node_activation,all_wgts)
    display(ui,inter)

## A) Understanding Activation Functions:

An <b>activation</b> (or <b>transfer</b> ) function converts the weighted sum of input values into an output value for a perceptron (you may want to look back at Activity 2 (A) for the perceptron model). 

There are several good choices of activation functions for ANNs which are biology inspired and work well with ANNs. Some of them are shown below in the diagram.
![ANN fig 5 ](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig5.png)
The MODIFY AND RUN CELL below has code for an activation function of a <b>[3,1]</b> ANN i.e. one with 3 inputs and 1 output. It shows the graphical output of the function as computed by the ANN built in PyTorch in one of the earlier code cells. Through widgets you will be able to directly modify the weights of this ANN (in practice the weights are "learned" which is something we will see later).

For answer questions 1 - 15 you will need to modify and run the code cell below multiple times. Be <b>attentive</b> to the hints in comments.


In [ ]:
#@title #(MODIFY AND RUN CELL) Coding for Activity 3 (A) to answer Questions 1 - 17
#@markdown <b>INSTRUCTION</b>: have ONLY one line uncommented at a time, i.e. no hashtag (#) in front of only one out_value computing line
def my_activation(in_value):
    out_value = in_value/2.0 # Function F
    #out_value = torch.sign(in_value) # Function G
    #out_value = torch.tanh(in_value) # Function H
    #out_value = torch.sign(in_value - 1.5) # Mystery Function X
    #out_value = torch.nn.functional.relu(in_value) # Mystery Function Y
    return out_value

make_output_node_widgets(ANN_model, ANN_model.fc1.in_features, ANN_model.fc1.out_features)

In [ ]:
#@title #(RUN upon completion) Activity 3 Questions 1 - 17
#@markdown <b>For Function F (make sure the correct statement is uncommented in the code cell above to plot function):</b>
#@markdown 1. If the sum of inputs is 2, what is the output?
activity3_answer1 = "" #@param {type:"string"}
#@markdown 2. If the sum of inputs is 1, what is the output?
activity3_answer2 = "" #@param {type:"string"}
#@markdown 3. What is the minimum output value for any input?
activity3_answer3 = "" #@param {type:"string"}
#@markdown 4. Could a small change in one input cause a large change in the output? 
activity3_answer4 = "" #@param {type:"string"}

#@markdown <b>For Function G (make sure the correct statement is uncommented in the code cell above to plot function):</b>
#@markdown 5. If the sum of inputs is 2, what is the output?
activity3_answer5 = "" #@param {type:"string"}
#@markdown 6. If the sum of inputs is 1, what is the output?
activity3_answer6 = "" #@param {type:"string"}
#@markdown 7. What is the minimum output value for any input?
activity3_answer7 = "" #@param {type:"string"}
#@markdown 8. Could a small change in one input cause a large change in the output? 
activity3_answer8 = "" #@param {type:"string"}

#@markdown <b>For Function H (make sure the correct statement is uncommented in the code cell above to plot function):</b>
#@markdown 9. If the sum of inputs is 2, what is the output?
activity3_answer9 = "" #@param {type:"string"}
#@markdown 10. If the sum of inputs is 1, what is the output?
activity3_answer10 = "" #@param {type:"string"}
#@markdown 11. What is the minimum output value for any input?
activity3_answer11 = "" #@param {type:"string"}
#@markdown 12. Could a small change in one input cause a large change in the output? 
activity3_answer12 = "" #@param {type:"string"}

#@markdown <b>Which of the activation functions (F,G,H) is:</b>
#@markdown 13. A step function?
activity3_answer13 = "" #@param {type:"string"}
#@markdown 14. A linear function?
activity3_answer14 = "" #@param {type:"string"}
#@markdown 15. A sigmoid function?
activity3_answer15 = "" #@param {type:"string"}

#@markdown 16. How would you describe mystery function X?
activity3_answer16 = "" #@param {type:"string"}
#@markdown 17. How would you describe mystery function Y?
activity3_answer17 = "" #@param {type:"string"}


# Activity 4. Perceptrons for Logic

In [ ]:
#@title #(RUN CELL) Class and Object for a [2,1] ANN creation in PyTorch suitable for Logic functions

class MyANNModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(2, 1,bias=False)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.fc1(x)
    x = my_activation(x)
    return x
    
ANN_model = MyANNModel()

In [ ]:
#@title #(RUN CELL) Helper Functions to create widgets for controlling the [2,1] PyTorch ANN

def make_output_node_widgets(model, num_input_nodes, out_node_num):
    wght_wdgts = [] # weight widgets
    inpt_wdgts = [] # input widgets
    for i in range(1,(num_input_nodes+1)):
        inpt_wdgts.append(widgets.FloatSlider(min=0,max=1,step=1,value=0,description='x'+str(i)))
        wght_wdgts.append(widgets.FloatSlider(min=0,max=2,step=1,value=-1,description='w'+str(i)+str(out_node_num)))
    ui = widgets.HBox([widgets.VBox(inpt_wdgts), widgets.VBox(wght_wdgts)])
    all_wgts ={}
    all_wgts['out_node_num'] = widgets.fixed(out_node_num)
    all_wgts['num_input_nodes'] = widgets.fixed(num_input_nodes)
    all_wgts['model'] = widgets.fixed(model)
    for xws in inpt_wdgts:
        all_wgts[xws.description] = xws
    for wws in wght_wdgts:
        all_wgts[wws.description] = wws
    inter = widgets.interactive_output(plot_output_node_activation,all_wgts)
    display(ui,inter)

## A) Thinking Logically with ANNs:

Logical thinking is ingrained in human language and patterns of communication. Can we create a simple [2,1] ANN to create a "logically thinking" ANN to reproduce the input/output functionality of the AND and OR functions shown below? 

Note that the AND and OR logic need to be implemented by two separate set of choices for weights. The inputs x1 and x2 are limited to 0 (for F) and 1 (for T). The weights can be 0, 1 or 2. <b>These constraints are set in how the sliders operate to select those values.</b>   
![ANN fig 6](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig6-logic.png)
![ANN fig 7](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig7-logic.png)


In [ ]:
#@title #(RUN CELL) Coding for Activity 4 (A) to answer Questions 1 - 2
def my_activation(in_value):
    out_value = torch.sign(torch.relu(in_value - 1))
    return out_value

make_output_node_widgets(ANN_model, ANN_model.fc1.in_features, ANN_model.fc1.out_features)

In [ ]:
#@title #(RUN upon completion) Activity 4 (A) Questions 1 - 2

#@markdown 1. What set of weights did you choose to implement the AND logic?
activity5_answer1 = "" #@param {type:"string"}
#@markdown 2. What set of weights did you choose to implement the OR logic?
activity5_answer2 = "" #@param {type:"string"}


# Activity 5. Perceptrons for Images

In [ ]:
#@title #(RUN CELL) Class and Object for a [9,4] ANN creation in PyTorch for 3 X 3 image input and 4 classes

class MyANNModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(9, 4,bias=False)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.fc1(x)
    x = my_activation(x)
    return x
    
ANN_model = MyANNModel()

In [ ]:
#@title #(RUN CELL) Helper Functions to create widgets for controlling the [9,4] PyTorch ANN
def plot_output_node_activation(model, num_input_nodes, out_node_num, **k):
    '''Plots the activation output of one node given a model which has no hidden layers'''
    plt.figure(figsize=(7,5))
    plt.title('Output of activation function from node ' + str(out_node_num))
    plt.ylim(-1.2, 1.2)
    plt.xlim(-2, 2)
    wsums = np.arange(-2,2,0.1)
    pl=plt.plot(wsums,my_activation(torch.tensor(wsums)))
    x_vals = []
    w_vals = []
    for i in range(num_input_nodes):
        x_vals.append(k['x'+str(i+1)])
        w_vals.append(k['w'+str(i+1)+str(out_node_num)])
        model.fc1.__dict__['_parameters']['weight'][out_node_num-1][i] = k['w'+str(i+1)+str(out_node_num)]
    xv = np.dot(x_vals,w_vals)
    yv = model.forward(torch.tensor([x_vals]))[0][out_node_num].item()
    plt.scatter([xv], [yv], color='red', s=30)
    plt.xlabel('weighted sum of inputs')
    plt.ylabel('activation function output')
    plt.grid(True)
    plt.tight_layout() 

def make_output_node_widgets(model, num_input_nodes, out_node_num):
    wght_wdgts = [] # weight widgets
    inpt_wdgts = [] # input widgets
    for i in range(1,(num_input_nodes+1)):
        inpt_wdgts.append(widgets.FloatSlider(min=0,max=1,step=1,value=0,description='x'+str(i)))
        wght_wdgts.append(widgets.FloatSlider(min=-1,max=1,value=-1,description='w'+str(i)+str(out_node_num)))
    ui = widgets.HBox([widgets.VBox(inpt_wdgts), widgets.VBox(wght_wdgts)])
    all_wgts ={}
    all_wgts['out_node_num'] = widgets.fixed(out_node_num)
    all_wgts['num_input_nodes'] = widgets.fixed(num_input_nodes)
    all_wgts['model'] = widgets.fixed(model)
    for xws in inpt_wdgts:
        all_wgts[xws.description] = xws
    for wws in wght_wdgts:
        all_wgts[wws.description] = wws
    inter = widgets.interactive_output(plot_output_node_activation,all_wgts)
    display(ui,inter)

In [ ]:
#@title # (RUN CELL) Visualize the [9,4] ANN
network=ANN_Arch_View([9,4]) # TRY CHANGING, the first number in this list is number of input nodes, next number is number of output nodes
network.draw(show_weights=True)

# A) Simple ANN to classify 3 X 3 optical character

ANNs have many applications, including computer vision and image recognition. You may have used computer software that is able to scan printed pages and convert them to digitized form to make them searchable. In order for the software to do this it has to understand the character represented in digital images which are tables of pixels. These automated tasks have different names like optical character recognition, handwriting recognition. Inside such software you will find ANNs which are more complex than the one you will build here, but those ANNs have similar building blocks. This section explores an example with 3x3 pixel black-and-white images, as show below. These 9 pixels will be the inputs (x1,...,x9) for an ANN, with outputs for each of 4 symbols (X, 十 , T, L). 

For each symbol, choose a set of 9 input weights and a step function threshold
to produce the correct output. All of the weights can be either +1.0 or -1.0 as constrained by the widgets below and pixels are either black (0) or white (1). In other words you have to answer questions like:

What set of 9 weights to choose to make the output node 1 fire for symbol X at the input?

What set of 9 weights to choose to make the output node 2 fire for symbol 十 at the input?

... and so on ... 

Suggestions: start by choosing an output node number from the 1, 2, 3, 4 which corresponding to the 4 symbols (X, 十 , T, L). Then configure the input pixels to be that input symbol and then choose a proper set of 9 weights so that the output node fires.

![ANN fig 9 ](https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/fig8.png)


In [ ]:
#@title #(MODIFY AND RUN CELL) Coding for Activity 5 (A) to answer Questions 1 - 5
#@markdown <b>INSTRUCTION:</b> modify value of the output node number as shown in code below to get different output nodes 
def my_activation(in_value):
    out_value = torch.sign(in_value)
    return out_value

output_node_num = 1 # TRY CHANGING, can be 1, 2, 3 or 4 corresponding to the 4 symbols (X, 十 , T, L)
make_output_node_widgets(ANN_model, ANN_model.fc1.in_features, output_node_num)

In [ ]:
#@title #(RUN upon completion) Activity 5 (A) Questions 1 - 5

#@markdown 1. What set of weights did you choose to make the output node 1 fire for symbol X?
activity5_answer1 = "" #@param {type:"string"}
#@markdown 2. What set of weights did you choose to make the output node 1 fire for symbol +?
activity5_answer2 = "" #@param {type:"string"}
#@markdown 3. What set of weights did you choose to make the output node 1 fire for symbol T?
activity5_answer3 = "" #@param {type:"string"}
#@markdown 4. What set of weights did you choose to make the output node 1 fire for symbol L? 
activity5_answer4 = "" #@param {type:"string"}
#@markdown 5. This approach could be used for larger images (more pixels), and with more outputs (for more symbols). Explain why these sorts of ANNs are called classifiers.
activity5_answer5 = "" #@param {type:"string"}

# B) Future Activity: teaching an ANN

In future activities we will explore algorithms to adjust weights so that ANNs can learn.
Learning (also called training ) usually requires a set of sample inputs.
When each input should give a known output, it is supervised learning .
When inputs are provided but the outputs are unknown, it is unsupervised learning.

In [ ]:
#@title #(RUN upon completion) Activity 5 (A) Questions 6 - 9
#@markdown Categorize each example below as supervised or unsupervised learning.
#@markdown 6. Given images of faces, identify the most common faces.
activity5_answer6 = "" #@param {type:"string"}
#@markdown 7. Given images of specific people, learn to recognize each person.
activity5_answer7 = "" #@param {type:"string"}
#@markdown 8. Given sets of test results for many patients, identify patients with similar symptoms.
activity5_answer8 = "" #@param {type:"string"}
#@markdown 9. Given sets of test results for many patients, provide the right diagnosis for each patient. 
activity5_answer9 = "" #@param {type:"string"}

In [ ]:
#@title Team Work Evaluation for the ANN notebook based activities.
#@markdown 1.	How much time was required for completion of the ANN notebook?
activity1_evaluation1 = "" #@param {type:"string"}
#@markdown 2.	Was the contribution from each participant equal?	
activity1_evaluation2 = "" #@param {type:"string"}
#@markdown 3.	How could the team work and learn more effectively?
activity1_evaluation3 = "" #@param {type:"string"} 
#@markdown 4.	How many participants thought the problems were too simple (trivial)? 
activity1_evaluation4 = "" #@param {type:"string"}
#@markdown 5.	How many participants thought the problems were at the proper level of difficulty?	
activity1_evaluation5 = "" #@param {type:"string"}
#@markdown 6.	How many participants thought the problems were too hard?
activity1_evaluation6 = "" #@param {type:"string"}
#@markdown 7.	Was help needed? Where?
activity1_evaluation7 = "" #@param {type:"string"}
#@markdown 8.	Does the team have any suggestions about how the ANN notebook could be improved? If so, how?
activity1_evaluation8 = "" #@param {type:"string"} 

# A Project With Real Images
# Code cells are ready to run (note that execution may be long ...)
# Discuss with instructor for ideas on what can be done

This is a project to train an ANN to classify fashion images. See below for samples.   
![alt text](https://torchfusion.readthedocs.io/en/latest/_images/fmnist.png)

In [ ]:
#@title # (RUN CELL) LIBRARY INSTALL COMMAND
!pip3 install --upgrade torchfusion

In [ ]:
#@title # (RUN CELL) TRAINING CODE
from torchfusion.layers import *
from torchfusion.datasets import *
from torchfusion.metrics import *
import torch.nn as nn
import torch.cuda as cuda
from torch.optim import Adam
from torchfusion.learners import StandardLearner

train_loader = fashionmnist_loader(size=28,batch_size=32)
test_loader = fashionmnist_loader(size=28,train=False,batch_size=32)

model = nn.Sequential(
    Flatten(),
    Linear(784,100),
    Swish(),
    Linear(100,100),
    Swish(),
    Linear(100,100),
    Swish(),
    Linear(100,10)
)

if cuda.is_available():
    model = model.cuda()

optimizer = Adam(model.parameters())

loss_fn = nn.CrossEntropyLoss()

train_metrics = [Accuracy()]
test_metrics = [Accuracy()]

learner = StandardLearner(model)

if __name__ == "__main__":

    print(learner.summary((1,28,28)))
    learner.train(train_loader,train_metrics=train_metrics,optimizer=optimizer,loss_fn=loss_fn,test_loader=test_loader,test_metrics=test_metrics,num_epochs=40,batch_log=False)

In [ ]:
#@title # (RUN CELL) TRAINING CODE
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/sbhattac/ai-workshop/master/ANN/"
for filename in ("sample-1.jpg", "sample-2.jpg", "sample-3.jpg", "sample-4.jpg"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + filename
    urllib.request.urlretrieve(url, filename)

In [ ]:
#@title # (RUN CELL) TESTING CODE
import torch
from torchfusion.layers import *
import torch.nn as nn
import torch.cuda as cuda
from torchfusion.learners import StandardLearner
from torchfusion.utils import load_image

model = nn.Sequential(
    Flatten(),
    Linear(784,100),
    Swish(),
    Linear(100,100),
    Swish(),
    Linear(100,100),
    Swish(),
    Linear(100,10)
)

if cuda.is_available():
    model = model.cuda()


#learner = StandardLearner(model)
#learner.load_model("best_models\model_20.pth")

if __name__ == "__main__":

    #map class indexes to class names
    class_map = {0:"T-Shirt",1:"Trouser",2:"Pullover",3:"Dress",4:"Coat",5:"Sandal",6:"Shirt",7:"Sneaker",8:"Bag",9:"Ankle Boot"}

    #Load the image
    image = load_image("sample-1.jpg",grayscale=True,target_size=28,mean=0.5,std=0.5)

    #add batch dimension
    image = image.unsqueeze(0)

    #run prediction
    pred = learner.predict(image)

    #convert prediction to probabilities
    pred = torch.softmax(pred,0)

    #get the predicted class
    pred_class = pred.argmax().item()

    #get confidence for the prediction
    pred_conf = pred.max().item()

    #Map class_index to name
    class_name = class_map[pred_class]
    print("Predicted Class: {}, Confidence: {}".format(class_name,pred_conf))